In [65]:
#Imports
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext
import os
import threading 
import time
from sqlalchemy import create_engine, Column, Integer, String, Text, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship, declarative_base
from sqlalchemy.exc import SQLAlchemyError
import atexit

# Function to handle chatbot summarization
from langchain_community.document_loaders import JSONLoader
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter, RecursiveJsonSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import OpenAIEmbeddings
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from dotenv import load_dotenv
load_dotenv()


True

In [66]:
#Chat History Database setup
DATABASE_URL = "sqlite:///chat_history_testTest.db"
Base = declarative_base()

class Session(Base):
    __tablename__ = "sessions"
    id = Column(Integer, primary_key=True)
    session_id = Column(String, unique=True, nullable=False)
    messages = relationship("Message", back_populates="session")

class Message(Base):
    __tablename__ = "messages"
    id = Column(Integer, primary_key=True)
    session_id = Column(Integer, ForeignKey("sessions.id"), nullable=False)
    role = Column(String, nullable=False)
    content = Column(Text, nullable=False)
    session = relationship("Session", back_populates="messages")

# Create the database and the tables
engine = create_engine(DATABASE_URL)
Base.metadata.create_all(engine)
SessionLocal = sessionmaker(bind=engine)

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


In [67]:
# Function to save a single message
def save_message(session_id: str, role: str, content: str):
    with SessionLocal() as db:
        try:
            session = db.query(Session).filter(Session.session_id == session_id).first()
            if not session:
                session = Session(session_id=session_id)
                db.add(session)
                db.commit()
                db.refresh(session)

            db.add(Message(session_id=session.id, role=role, content=content))
            db.commit()
        except SQLAlchemyError:
            db.rollback()


# Function to load chat history
def load_session_history(session_id: str) -> ChatMessageHistory:
    chat_history = ChatMessageHistory()
    with SessionLocal() as db:
        try:
            session = db.query(Session).filter(Session.session_id == session_id).first()
            if session:
                for message in session.messages:
                    chat_history.add_message({"role": message.role, "content": message.content})
        except SQLAlchemyError:
            pass

    return chat_history


# Modify the get_session_history function to use the database
def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        # Load from the database if not in store
        store[session_id] = load_session_history(session_id)
    return store[session_id]


# Ensure you save the chat history to the database when needed
def save_all_sessions():
    for session_id, chat_history in store.items():
        for message in chat_history.messages:
            save_message(session_id, message["role"], message["content"])

# Register function to save sessions before exit
atexit.register(save_all_sessions)

store = {}


In [68]:
def parentChatbotSummarize(filepath: str, session_id: str, promptQuestion: str):
    try:
        llm = ChatOpenAI(model='gpt-4o')

        loader = JSONLoader(file_path=filepath, jq_schema='.', text_content=False)
        docs = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=200, add_start_index=True)  # Increased chunk-size to 10000 from 1000
        splits = text_splitter.split_documents(docs)
        vectorstore = InMemoryVectorStore.from_documents(documents=splits, embedding=OpenAIEmbeddings())
        retriever = vectorstore.as_retriever(search_type='similarity')

        contextualize_q_system_prompt = (
            "Given a chat history and the latest user question "
            "which might reference context in the chat history, "
            "formulate a standalone question which can be understood "
            "without the chat history. Do NOT answer the question, "
            "just reformulate it if needed and otherwise return it as is."
        )

        contextualize_q_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", contextualize_q_system_prompt),
                MessagesPlaceholder("chat_history"),
                ("human", "{input}"),
            ]
        )

        history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)
        system_prompt = (
            "You are a nurse updating a parent on the status of their child in the NICU. "
            "Use the retrieved context to provide relevant information to the parent. "
            "Speak in a conversational tone, and keep the summary concise. "
            "Do not diagnose."
            "You always have access to the chat history IF NEEDED" #was needed to remember
            "\n\n{context}"
        )

        qa_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                MessagesPlaceholder("chat_history"),
                ("human", "{input}"),
            ]
        )
        question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
        rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

        conversational_rag_chain = RunnableWithMessageHistory(
            rag_chain,
            get_session_history,
            input_messages_key="input",
            history_messages_key="chat_history",
            output_messages_key="answer",
        )
        response = conversational_rag_chain.invoke(
            {"input": promptQuestion},
            config={
                "configurable": {"session_id": session_id}
            },  # constructs a key "abc123" in `store`.
        )

        # Save the user question and AI response to the database
        save_message(session_id, "human", promptQuestion)
        save_message(session_id, "ai", response['answer'])

        return response['answer']
    except Exception as e:
        return f"Error: {str(e)}"


In [69]:
def loading_dots():
    dot_count = 0
    while loading:
        dot_count = (dot_count % 3) + 1
        dots = ". " * dot_count
        chat_history_text.configure(state='normal')
        chat_history_text.delete('end-1c linestart', 'end-1c')  # Remove previous dots
        chat_history_text.insert(tk.END, dots)
        chat_history_text.configure(state='disabled')
        root.update_idletasks()
        time.sleep(0.5)


In [70]:
def run_chatbot():
    filepath = file_entry.get()
    session_id = session_entry.get()
    prompt = prompt_entry.get()

    if not os.path.exists(filepath):
        messagebox.showerror("Error", "The selected file does not exist.")
        return

    if not session_id:
        messagebox.showerror("Error", "Please enter a valid session ID.")
        return

    if not prompt:
        messagebox.showerror("Error", "Please enter a prompt.")
        return

    chat_history_text.configure(state='normal')
    chat_history_text.insert(tk.END, f"You: {prompt}\nBot: ")
    chat_history_text.configure(state='disabled')
    root.update_idletasks()

    global loading
    loading = True
    threading.Thread(target=loading_dots).start()

    def generate_response():
        global loading
        result = parentChatbotSummarize(filepath, session_id, prompt)
        loading = False
        chat_history_text.configure(state='normal')
        chat_history_text.delete('end-1c linestart', 'end-1c')  # Remove loading dots
        chat_history_text.insert(tk.END, f"{result}\n\n")
        chat_history_text.configure(state='disabled')
        prompt_entry.delete(0, tk.END)

    threading.Thread(target=generate_response).start()


In [71]:
def end_chat():
    chat_history_text.configure(state='normal')
    chat_history_text.insert(tk.END, "Chat session ended.\n\n")
    chat_history_text.configure(state='disabled')
    session_entry.delete(0, tk.END)
    prompt_entry.delete(0, tk.END)
    file_entry.delete(0, tk.END)
    root.after(1000, root.destroy)  # Close the GUI after 1 second


In [72]:
# GUI Setup using Tkinter
root = tk.Tk()
root.title("Chatbot GUI for JSON Summarization")
root.geometry("1000x1000")

# Filepath Entry
file_label = tk.Label(root, text="Select JSON File:")
file_label.pack(pady=5)
file_entry = tk.Entry(root, width=50)
file_entry.pack(pady=5)
file_button = tk.Button(root, text="Browse", command=lambda: file_entry.insert(0, filedialog.askopenfilename(filetypes=[("JSON Files", "*.json")])))
file_button.pack(pady=5)

# Session ID Entry
session_label = tk.Label(root, text="Enter Session ID:")
session_label.pack(pady=5)
session_entry = tk.Entry(root, width=50)
session_entry.pack(pady=5)

# Prompt Entry
prompt_label = tk.Label(root, text="Enter Prompt:")
prompt_label.pack(pady=5)
prompt_entry = tk.Entry(root, width=50)
prompt_entry.pack(pady=5)

# Chat History Display
chat_history_text = scrolledtext.ScrolledText(root, width=120, height=40, state='disabled', wrap='word')
chat_history_text.pack(pady=10)

# Run Button
run_button = tk.Button(root, text="Run Chatbot", command=run_chatbot)
run_button.pack(pady=5)

# End Chat Button
end_button = tk.Button(root, text="End Chat", command=end_chat)
end_button.pack(pady=5)

# Mainloop to run the GUI
root.mainloop()
